# Optimization of compute-bound Python code 

## Layout:

* Foreword about optimization and profiling:
    - No optimisation without profiling
    - No benchmarking without tests
* Presentation of the example:
    - Square crystal: demonstration
    - Relaxed circular crystal: exercise
* `Python` version:
    - code with loops à la `FORTRAN`
    - Bottleneck search
* `Numpy` vectorized version:
    - Limits of vectorization: cache sizes
* A word on threading and multi-processing
[...]

* `Numexpr` compiler for mathematical formula
    - Single and multi-threaded evaluation
    - Limits of `numexpr`
* `Numba`: just in time compilation of numerical code
    - Decoration of function
    - Type annotation
    - Limits of `numba`
* `Cython`: ahead of time compilation
    - `Cython` usage within `Jupyter`
    - Annotated compilation 
    - Single threaded performances
    - Parallelization with OpenMP
    - Mixing extensions with Python threads
* Conclusions:
    - Only addresses CPU-bound issues, not I/O issues
    - Limits of parallelization: [Amdahl's law](https://en.wikipedia.org/wiki/Amdahl%27s_law)
    - If your problem is still compute-bound, then move to GPU

## Definitions:
* In computer science, software **optimization** is the process of modifying a software system to make some aspect of it work more efficiently or use fewer resources.[[Wikipedia](https://en.wikipedia.org/wiki/Program_optimization)]
* In software engineering, software **profiling** is a form of dynamic program analysis that measures, for example, the space (memory) or time complexity of a program, the usage of particular instructions, or the frequency and duration of function calls. Most commonly, profiling information serves to aid program optimization, and more specifically, performance engineering. [[Wikipedia](https://en.wikipedia.org/wiki/Profiling_%28computer_programming%29)]
* Regression **testing** is re-running functional and non-functional tests to ensure that previously developed and tested software still performs after a change. If not, that would be called a regression. [[Wikipedia](https://en.wikipedia.org/wiki/Regression_testing)]

## Methodology:

1. **Make your algorithm work**, regardless how you implement it.
2. Make it right and ensure it is right before writing **non-regression tests**.
3. **Benchmark** it. 
4. If it fast enough, goto 9.
5. **Profile** it to see where most of the time is spent.
6. **Optimize** the code where most time is spent. 
7. **Check** non-regression tests are still passing:
   * Fast algorithms giving wrong answers are useless!
8. **Loop** to 3.
9. Your work is **done** and you deserve a 🍺 !

**Premature optimization is the root of all evil (or at least most of it) in programming.** *Donald Knuth*

## Profiling in Python

The standard library of *Python* contains the `cProfile` module which collects information about the running code. It is a *C* version of the `profile` module.

Profile results can be analyzed with the `pstats` module or exernal programs like [**RunSnakeRun**](http://www.vrplumber.com/programming/runsnakerun/), [**SnakeViz**](https://jiffyclub.github.io/snakeviz/) or [**kcachegrind**](https://kcachegrind.github.io/) (using [pyprof2calltree](https://pypi.org/project/pyprof2calltree/) converter). 

*Profiling* induces slow-down, it should be distinguished from *benchmarking* which measures the actual performances.

Here is the link to the official Python documentation: https://docs.python.org/3/library/profile.html

In [ ]:
import math, cProfile

prf = cProfile.Profile()
prf.enable() # Start of the profiled section:
size = 1000000
distance = 1
tth = []
for i in range(size):
    x_i = size//1000/1000
    y_i = size%1000/1000
    tth.append(math.atan2(math.sqrt(x_i**2 + y_i**2), distance))

prf.disable() #End of section:

prf.dump_stats("profile.log"); prf.print_stats(sort=1) # Save / display some results

Example of visualization of the profiling with **runSnakeRun** or **SnakeViz**. Equivalent visualization is available from the other tools.

This is the typical way of using the Python profiler. Another way to use it is to invoke it when starting the process:

`python -m cProfile -o profile.log myscript.py`


In [ ]:
#!snakeviz profile.log

## Jupyter magic functions for profiling

Line magics:

* `%time`: measure the time to run one statement/function call, once
* `%timeit`: measure in benchmarking mode (many repetitions, garbage collector disabled)
* `%prun`: run command within the profiler
* `%lprun`: run with the line-profiler active
* `%memit`: check for memory allocations
* `%mprun`: run with memory profile, line per line

All those functions can be used with `%%` to apply to the full cell instead of a single command.
    
Mind to ask for their help ... sometimes they have multiple options `%timeit?`

In [ ]:
%timeit?

In [ ]:
# Description of the system:
import os
import cpuinfo

if hasattr(os, 'sched_getaffinity'):  # Some Unix only
    # Get the number of cores the Python process has access to
    # This provides the number of cores requested in SLURM
    n_cpu = len(os.sched_getaffinity(0))
else:
    n_cpu = os.cpu_count()
n_total_cpu = os.cpu_count()

print(f"We can use {n_cpu} out of the {n_total_cpu} cores of the computer")
# Limit the number of cores we use to at most 4
n_cpu = min(4, n_cpu); print(f"We are using only {n_cpu} cores")

for key, val in cpuinfo.get_cpu_info().items():
    if "cache_size" in key:
        print(key, val)

## Two-dimensional Laue function example

All this training is based on the same example: 2D diffraction from a finite size crystallite given by Laue function:

$$ I(H,K) = \left | \sum_{n=0}^{N-1} \sum_{m=0}^{N-1} \exp \left [ 2 \pi i \left ( H n + K m \right ) \right ] \right| ^2 $$

This equation describes the scattering from a square crystal:
- **N** is the number of unit cells of the crystal in both directions
- **n** and **m** are the indices of the unit-cell in each direction (**real space** coordinates),
- **H** and **K** are the continuous Miller indices (**reciprocal space** coordinates).

This example is derived from: https://journals.iucr.org/j/issues/2019/04/00/gj5229/gj5229.pdf

$$ I(H,K) = \left | \sum_{n=0}^{N-1} \sum_{m=0}^{N-1} \exp \left [ 2 \pi i \left ( H n + K m \right ) \right ] \right| ^2 $$

In [ ]:
import numpy as np
import cmath

def laue(N, H, K):
    """Laue formulae for a square crystal with N² unit cells given H, K coordinate"""
    tmp = 0.0
    for n in range(N):  # loop and sum over unit-cells
        for m in range(N):
            tmp += cmath.exp(2j*np.pi*(H*n + K*m))
    return abs(tmp)**2

laue(N=32, H=0, K=4)

To build an image of the diffraction peak, the continuous Miller indices are sampled in the neighborhood of **H**, **K**:

- $h \in [H-0.5,H+0.5]$
- $k \in [K-0.5,K+0.5]$

With $oversampling * N$ samples on each direction, where:

- **N** is the number of unit cells of the crystal in both directions
- **oversampling** defines how many points are needed to describe a single Laue fringe (2 = Nyquist frequency)

### Representation of the square crystal:
<img src="square_crystal_coordinates.png" alt="Square crystal coordinates" width="300">

### Visualization of the diffraction peak for a square 2D crystal around H=0, K=4:
![bragg peak](square_crystal_bragg_peak.png)

In [ ]:
%matplotlib inline 
# This is for plotting
import math
import cmath
import numpy as np
from matplotlib.pyplot import subplots
from matplotlib.colors import LogNorm

In [ ]:
# Miller index of reflection
H = 0 
# Miller index of reflection
K = 4 
# Number of unit cells per direction
N = 32 
# Defines how many points are needed to describe a single Laue fringe (2 = Nyquist frequency)
oversampling = 3

# Generate real (n,m) and reciprocal (h, k) space coordinates
# Generate real and reciprocal space coordinates
n = np.arange(N)
m = np.arange(N)
h = np.arange(H-0.5, H+0.5, 1./(oversampling*N))
k = np.arange(K-0.5, K+0.5, 1./(oversampling*N))

In [ ]:
def laue_python(N, h, k):
    result = np.zeros((h.size, k.size))
    for i_h, v_h in enumerate(h):  # loop over the reciprocal space coordinates
        for i_k, v_k in enumerate(k):
            tmp = 0.0
            for n in range(N):  # loop and sum over unit-cells
                for m in range(N):
                    tmp += cmath.exp(2j*np.pi*(v_h*n + v_k*m))
            result[i_h, i_k] = abs(tmp)**2
    return result

#Compute
%time square_intensity = laue_python(N, h, k)

In [ ]:
# Visualization of the reference images
fig, ax = subplots(1,2, figsize=(10,5))
fig.suptitle("Bragg peak")
ax[0].imshow(np.load("reference_sq.npy").T, extent=(h.min(), h.max(), k.min(), k.max()), norm=LogNorm(), origin = 'lower')
ax[0].set_xlabel('H')
ax[0].set_ylabel('K')
ax[0].set_title(f"Square crystal {N}x{N}")
ax[1].imshow(np.load("reference_ci.npy").T, extent=(h.min(), h.max(), k.min(), k.max()), norm=LogNorm(), origin = 'lower')
ax[1].set_xlabel('H')
ax[1].set_ylabel('K')
ax[1].set_title(f"Circular crystal {N}x{N}")
pass

### Validation of optimization

Code optimization should always be performed with some tests to ensure the speed-up obtained is not degrading the numerical quality !

In [ ]:
def validate_sq(result):
    "Return the error value"
    reference = np.load("reference_sq.npy")
    return abs(reference-result).max()/reference.max()

def validate_ci(result):
    "Return the error value (for exercises)"
    reference = np.load("reference_ci.npy")
    return abs(np.array(reference)-result).max()/reference.max()

def display(result):
    "Display the array"
    fig, ax = subplots()
    fig.suptitle("Bragg peak")
    ax.imshow(result.T, extent=(h.min(), h.max(), k.min(), k.max()), norm=LogNorm(), origin = 'lower')
    ax.set_xlabel('H');ax.set_ylabel('K')
    ax.set_title(f"Crystal {N}x{N}")
    
print("Error measured:", validate_sq(square_intensity))  # Validation of the result:
perf_sq_python = %timeit -o laue_python(N, h, k)         # Measure performances of the Python implementation

In [ ]:
# Run a function in the Python profiler ...
%prun -D square.log laue_python(N, h, k)

In [ ]:
# Line profiler:
%load_ext line_profiler
%lprun -f laue_python laue_python(N, h, k)

In [ ]:
#Memory profiler
%load_ext memory_profiler
%memit laue_python(N, h, k)

## Exercise


### Circular crystal

Let's first consider the case of a **circular** crystal rather than a square:
1. Write a function generating the diffraction from a **circular** 2D crystal without strain.
The previous formula then becomes:
   $$ I(H,K) = \left | \sum_{n=0}^{N-1} \sum_{m=0}^{N-1} \Omega \left (n,m \right ) \exp \left [ 2 \pi i \left ( H n + K m \right ) \right ] \right | ^2$$

   With $R=N/2$ the radius of the disk, the support function $\Omega\left (n,m \right )$ is defined as: 
     * $\Omega \left (n,m \right ) = 0$  where $(n-N/2)^2+(m-N/2)^2>(N/2)^2$
     * $\Omega \left (n,m \right ) = 1$  where $(n-N/2)^2+(m-N/2)^2 \leq (N/2)^2$

2. How do these modifications affect the result? the execution speed?

### Visualization of a circular 2D crystal:
<img src="circular_crystal_coordinates.png" alt="Circular crystal coordinates" width="400">

### Strained circular crystal

Let's consider the case where the circular crystal atoms are not perfectly ordered (**strain**).

**Strain** implies that unit-cells are shifted from their regular position according to $\overrightarrow{r'_i} = \overrightarrow{r_i} + \overrightarrow{\Delta r_i}$, where $\overrightarrow{\Delta r_i}$ is the displacement for the cell *i*.
For the case of a circular crystal, a function describing a progressive dilatation when moving from the center towards the periphery. 
It can be written:

$$
\overrightarrow{\Delta r}(\overrightarrow{r}) = e_{0} \overrightarrow{r} \left(1 + \tanh\left(\frac{r-R}{w}\right)\right)
$$
where:

- $\overrightarrow{\Delta r}(\overrightarrow{r})$ is the displacement of the unit-cell located at a distance $r$ from the crystal center,
- $R=N/2$ is the radius of the disk,
- $e_0$ is the maximum strain,
- $w$ is a parameter describing the width of the displacement profile (for small values of $w$ the strain is confined at the periphery of the crystal, whereas large values also affect the interior of the crystal).

### Visualization of the strain of the crystal, as colormap and along a diameter:
![Circular crystal strain map](circular_crystal_strain_map.png)

![Circular crystal strain map2](circular_crystal_strain_map2.png)

The previous formula then becomes:

$$ I(H,K) = \left | \sum_{n=0}^{N-1} \sum_{m=0}^{N-1} \Omega \left (n,m \right ) \exp \left \{ 2 \pi i \left [ H \left (n + \Delta n_{n,m}\right )+ K \left (m + \Delta m_{n,m}\right ) \right ] \right \} \right | ^2$$

Where
$$
\Delta n_{n,m} = e_0 (n - N/2) \left(
    1 + \tanh\left(
        \dfrac{r(n, m) - N/2}{w}
    \right)
\right)
$$
  
$$
r(n, m) = \sqrt{(n - N/2)^2 + (m - N/2)^2}
$$


**Exercise**:

1. Modify the function to take into account the displacement induced by the strain.
2. How these changes affect the result? the execution speed? Store the timimg in `perf_ci_python`

In [ ]:
# Some constants valid for all calculations
# Miller index of reflection
H = 0 
# Miller index of reflection
K = 4 
# Number of unit cells per direction
N = 32 
# Defines how many points are needed to describe a single Laue fringe (2 = Nyquist frequency)
oversampling = 3

# Radius of the crystal
R = N/2

# Maximum strain at surface
e0 = 0.01 
# Width of the strain profile below the surface
w = 5.

# Generate real and reciprocal space coordinates
n = np.arange(N)
m = np.arange(N)
h = np.arange(H-0.5, H+0.5, 1./(oversampling*N))
k = np.arange(K-0.5, K+0.5, 1./(oversampling*N))

In [ ]:
# Displacement of atoms as function of the radius
def delta(radius, crystal_radius, strain_width):
    """Displacement of atoms as function of the radius"""
    return 1 + np.tanh((radius - crystal_radius) / strain_width)

fig, ax = subplots()
x = np.linspace(0, R, 1000)
ax.plot(x, delta(x, R, w))
ax.set_xlabel("Radius")
ax.set_ylabel("Displacement")
t = ax.set_title("Strain as function of radius")

1. Implement the circular crystal **without** strain.

In [ ]:
from solution import circ_python_1   # Replace with your implementation
# def circ_python_1(N, h, k):
#    ...

In [ ]:
%time intensity_circle = circ_python_1(N, h, k)
print("Error:", validate_ci(intensity_circle))
display(intensity_circle)

2. Extend your code to add strain.

In [ ]:
from solution import circ_python   # Replace with your implementation
# def circ_python(N, h, k):
#    ...

In [ ]:
%time intensity = circ_python(N, h, k)
display(intensity)
perf_ci_python = %timeit -o circ_python(N, h, k)

3. Profile your strained crystal diffraction with `%prun` and `%lprun` and `%memit`.

In [ ]:
%prun -D circle.log circ_python(N, h, k)

In [ ]:
%lprun -f circ_python circ_python(N, h, k)

In [ ]:
%memit circ_python(N, h, k)

## Optimization with `NumPy`

Python is a dynamic language and is known to be pretty inefficient when looping over large datasets.
The `numpy` library offers vectorial notation which avoids those loops.

### Square crystal implemented with `NumPy`

This is how this code looks like once fully vectorized with `NumPy`:

In [ ]:
def laue_numpy(N, h, k):
    h = h.reshape(-1, 1, 1, 1)
    k = k.reshape(1, -1, 1, 1)
    n = np.arange(N).reshape(1, 1, -1, 1)
    m = np.arange(N).reshape(1, 1, 1, -1)
    return np.abs(np.sum(np.exp(2j * np.pi * (h*n + k*m)), axis=(2, 3)))**2

In [ ]:
def laue_numpy(N, h, k):
    h = h.reshape(-1, 1, 1, 1)
    k = k.reshape(1, -1, 1, 1)
    n = np.arange(N).reshape(1, 1, -1, 1)
    m = np.arange(N).reshape(1, 1, 1, -1)
    return np.abs(np.sum(np.exp(2j * np.pi * (h*n + k*m)), axis=(2, 3)))**2

# Compute and check error
%time intensity = laue_numpy(N, h, k)
print("Error:", validate_sq(intensity))
perf_sq_numpy = %timeit -o laue_numpy(N, h, k)

The first striking observation is that the code is much more concise: it can fit in a single line.
It is also much clearer: shorter and with a syntax similar to the provided formula.

Lines 2-5 add new (empty) dimensions to the input arrays. With this transformation, the calculation of `h*n + k*m` actually returns a 4-dimensional array. This important feature of NumPy is known as [**broadcasting**](https://numpy.org/doc/stable/reference/ufuncs.html#broadcasting).
The exponential then operates on all cells of this array.
The sum over the real-space coordinates is performed using numpy's `sum()` function: the `axis=(2, 3)` argument tells that the summation has to be performed over the last two dimensions of the array that contain the real space variables.
Besides the cleaner syntax, this implementation is also much faster than Python code.

In [ ]:
# Broadcasting example
a = np.array([1, 2, 3])
b = a.reshape(-1, 1)
print('a =', a)
print('b =', b)
print('a*b =', a*b)

### Exercise

1. Implement the strained crystal diffraction using `NumPy` ([Documentation](https://numpy.org/doc/stable/reference/index.html)).
2. Profile the code and store the timings in `perf_ci_numpy`.

In [ ]:
from solution import circ_numpy   # Replace with your implementation
# def circ_numpy(N, h, k):
#    ...

In [ ]:
#Compute and check error
%time intensity = circ_numpy(N, h, k)
print("Error:", validate_ci(intensity))
perf_ci_numpy = %timeit -o circ_numpy(N, h, k)

### Limits of `numpy`'s vectorization

When calculating expression like `a+2*b+5*c+a*b`, each individual operation creates a new temporary array where all results of each binary operation are stored. This can be an issue when the number of operands is large and that those temporary arrays are larger than the cache of the processor. Those temporary arrays need to go back to central memory before starting next operation and this is bad for performances.

In [ ]:
#Importance of the cache:
def tmp_numpy(N, h, k):
    h = h.reshape(-1, 1, 1, 1)
    k = k.reshape(1, -1, 1, 1)
    n = np.arange(N).reshape(1, 1, -1, 1)
    m = np.arange(N).reshape(1, 1, 1, -1)
    return np.exp(2j*np.pi*(h*n + k*m))
print(f"Size of the intermediate array: {tmp_numpy(N,h,k).nbytes/2**20:.2f} MiB")

In [ ]:
%memit laue_numpy(N, h, k)

## Multi-threading in Python

Threads are lightweight processes which have all access to the process memory and have some own local memory space. 
It looks like a good idea to have different threads performing some work in parallel to use the multi-cores of our modern processors. 

Python offers access to threads via the `threading` library, and offers some convenient tools like the `ThreadPool` from `multiprocessing.pool`, to apply the same function to a set of different inputs. 

In [ ]:
from multiprocessing.pool import ThreadPool
from itertools import product

def laue_mt(N, h, k):
    n = np.arange(N).reshape(-1, 1)
    m = np.arange(N).reshape(1, -1)
    def laue_st(hi, ki):
        return np.abs(np.exp(2j*np.pi*(hi*n + ki*m)).sum())**2
    with ThreadPool(n_cpu) as pool:
        tmp = pool.starmap(laue_st, product(h, k))
    return np.array(tmp).reshape(h.size, k.size)

%time intensity = laue_mt(N, h, k)
print("Error:", validate_sq(intensity))

In [ ]:
def laue_mt(N, h, k):
    n = np.arange(N).reshape(-1, 1)
    m = np.arange(N).reshape(1, -1)
    def laue_st(hi, ki):
        return np.abs(np.exp(2j*np.pi*(hi*n + ki*m)).sum())**2
    with ThreadPool(n_cpu) as pool:
        tmp = pool.starmap(laue_st, product(h, k))
    return np.array(tmp).reshape(h.size, k.size)

In [ ]:
perf_sq_mt = %timeit -o laue_mt(N, h, k)
print(f"Speed-up {perf_sq_numpy.best/perf_sq_mt.best:6.3f}x over {n_cpu} processes")

By running `htop` during the execution of the previous cell, one validates that many cores are used simultaneously but the execution time is much longer. The performances are ~50% worse than without threads!
All threads are **fighting for the Global Interpreter Lock (GIL)**, i.e., they are all waiting for the GIL to be allowed to run a bit of code.
This explains why, by going parallel, it is slower (with pure Python code).

### Introduction to the GIL

Indeed, using threads looks like a good idea but turns out to be more complicated than expected.
Having multiple threads accessing to the same Python object, possibly modifying them inplace, is dangerous. 
This is why developers of Python introduced the GIL (Global Intepreter Lock) which prevents multiple Python objects from being accessed simulaneously.

The GIL is at the core of C-Python but other implementations of Python do not suffer from the GIL: PyPy, Jython or IronPython are some examples. 
For now, there is no reason to hope this will change in the future. Just learn to live with the GIL!

There are two ways to work around the GIL:
1. Using separated processes
2. Use GIL-free sections

#### 1. Using separated processes
   Processes have distinct memory spaces and can run simultaneously without interfering. There is a `ProcessPool` 
   which works like the `ThreadPool` except that workers are separated processes. 
   The drawback of this method is that both the program, operands and the result should be sent to the worker and back.
   This requires them to be **serialized** and fully self-consistent. For example the shared variables *n* and *m* are no more allowed. 

The next cell presents an example of implementation:

In [ ]:
from multiprocessing.pool import Pool
from itertools import product

class LaueSP:
    def __init__(self, N):
        self.n = np.arange(N).reshape(1, -1)
        self.m = np.arange(N).reshape(-1, 1)
    def __call__(self, h, k):
        return np.abs(np.exp(2j*np.pi*(h*self.n + k*self.m)).sum())**2

def laue_mp(N, h, k):
    with Pool(n_cpu) as pool:
        laue = LaueSP(N)
        tmp = pool.starmap(laue, product(h, k))     
    return np.array(tmp).reshape(h.size, k.size)

In [ ]:
class LaueSP:
    def __init__(self, N):
        self.n = np.arange(N).reshape(1, -1)
        self.m = np.arange(N).reshape(-1, 1)
    def __call__(self, h, k):
        return np.abs(np.exp(2j*np.pi*(h*self.n + k*self.m)).sum())**2

def laue_mp(N, h, k):
    with Pool(n_cpu) as pool:
        laue = LaueSP(N)
        tmp = pool.starmap(laue, product(h, k))     
    return np.array(tmp).reshape(h.size, k.size)
%time intensity = laue_mp(N, h, k)
print("Error:", validate_sq(intensity)) #Compute and check error
perf_sq_mt = %timeit -o laue_mp(N, h, k)
print(f"Speed-up {perf_sq_numpy.best/perf_sq_mt.best:6.3f}x over {n_cpu} processes")

A class instance *laue* has to be used to ship the shared variable ($n$, $m$) to other processes, making the code a bit harder to read for people not used to object oriented programming (OOP).

Moreover, the speed-up obtained (2) is far from the resources consumed (4), this is due to the communication overhead.

#### 2. Use threads with section which are all **GIL-free**

   Next section will present three tools which are all compilers able to produce parallel code which is actually run using all cores of your computer.
   * `NumExpr`: Fast numerical expression evaluator
   * `Numba`: JIT Python compiler
   * `Cython`: C-extensions for Python
   
Several other tools exists (`Pythran`, `Transonic`, ...) but won't be presented.

In [ ]:
# Reload necessary packages, magics, functions and variables
%matplotlib inline 
%load_ext memory_profiler
%load_ext line_profiler

import os
import math
import cmath
import numpy as np
from matplotlib.pyplot import subplots
from matplotlib.colors import LogNorm

# Number of cores to use
if hasattr(os, 'sched_getaffinity'):  # Some Unix only
    # Get the number of cores the Python process has access to
    # This provides the number of cores requested in SLURM
    n_cpu = len(os.sched_getaffinity(0))
else:
    n_cpu = os.cpu_count()
n_cpu = min(8, n_cpu)


# Validation functions

def validate_sq(result):
    "Return the error value"
    reference = np.load("reference_sq.npy")
    return abs(reference-result).max()/reference.max()

def validate_ci(result):
    "Return the error value (for exercises)"
    reference = np.load("reference_ci.npy")
    return abs(np.array(reference)-result).max()/reference.max()

def display(result):
    "Display the array"
    fig, ax = subplots()
    fig.suptitle("Bragg peak")
    ax.imshow(result.T, extent=(h.min(), h.max(), k.min(), k.max()), norm=LogNorm(), origin = 'lower')
    ax.set_xlabel('H');ax.set_ylabel('K')
    ax.set_title(f"Crystal {N}x{N}")

    
# Constants
# Miller index of reflection
H = 0 
# Miller index of reflection
K = 4 
# Number of unit cells per direction
N = 32 
# Defines how many points are needed to describe a single Laue fringe (2 = Nyquist frequency)
oversampling = 3

# Radius of the crystal
R = N/2

# Maximum strain at surface
e0 = 0.01 
# Width of the strain profile below the surface
w = 5.

# Generate real and reciprocal space coordinates
n = np.arange(N)
m = np.arange(N)
h = np.arange(H-0.5, H+0.5, 1./(oversampling*N))
k = np.arange(K-0.5, K+0.5, 1./(oversampling*N))

## `NumExpr` is a fast numerical expression evaluator for `NumPy`.

It works best for evaluating large numerical expressions on arrays which do not fit into cache. The speed-up is typically of the order of 4-8x (may be limited by the number of cores since `numexpr` uses all cores).

In [ ]:
import numexpr as ne
ne.set_num_threads(n_cpu)  # Limit the number of threads to be used

a = np.arange(10)
ne.evaluate("sin(a)**2 + cos(a)**2")

How does `NumExpr` work:
1. Parse the mathematical expression 
2. Compile it in its virtual machine
2. Split data into chunks (4096 bytes: cache friendly). Broadcasting is handled as well.
3. Chunks can be processed in multiple threads

In [ ]:
# How does NumExpr actually work ...
nex = ne.NumExpr("sin(a)**2 + cos(a)**2")
print(ne.disassemble(nex))

### Square crystal implemented with `NumExpr`

* Only the complex exponential part is implemented in `NumExpr`.
* `NumExpr` implements efficiently the broadcast of data.
* Reductions (sum) exists but it is not faster than `NumPy`.

In [ ]:
def laue_numexpr(N, h, k):
    h = h.reshape(-1, 1, 1, 1)
    k = k.reshape(1, -1, 1, 1)
    n = np.arange(N).reshape(1, 1, -1, 1)
    m = np.arange(N).reshape(1, 1, 1, -1)
    j2pi = np.pi*2j
    tmp = ne.evaluate("exp(j2pi*(h*n + k*m))")
    return np.abs(np.sum(tmp, axis=(2, 3)))**2

#Compute and check error
%time intensity = laue_numexpr(N, h, k)
print("Error:", validate_sq(intensity))
perf_sq_numexpr = %timeit -o laue_numexpr(N, h, k)
%memit laue_numexpr(N, h, k)

In [ ]:
# Full NumExpr version ... not faster.
def laue_numexpr2(N, h, k):
    h = h.reshape(-1, 1, 1, 1)
    k = k.reshape(1, -1, 1, 1)
    n = np.arange(N).reshape(1, 1, -1, 1)
    m = np.arange(N).reshape(1, 1, 1, -1)
    j2pi = np.pi*2j
    tmp = ne.evaluate("exp(j2pi*(h*n + k*m))")
    tmp.shape = h.size, k.size, -1
    return abs(ne.evaluate("sum(tmp, axis=2)"))**2

#Compute and check error
intensity = laue_numexpr2(N, h, k)
print("Error:", validate_sq(intensity))
perf_sq_numexpr2 = %timeit -o laue_numexpr2(N, h, k)
%memit laue_numexpr2(N, h, k)

### Exercise

1. Implement the strained crystal diffraction using `NumExpr` ([User Guide](https://numexpr.readthedocs.io/projects/NumExpr3/en/latest/user_guide.html)).
2. Profile the code and store the timings in `perf_ci_numexpr`.

In [ ]:
from solution import circ_numexpr   # Replace with your implementation
# def circ_numexpr(N, h, k):
#    ...

In [ ]:
%time  intensity = circ_numexpr(N, h, k)
print("Error:", validate_ci(intensity))
perf_ci_numexpr = %timeit -o circ_numexpr(N, h, k)

### Limits of `NumExpr`
* Limited to mathematical functions
* Limited to element-wise evaluation (reduction is under development)

## `Numba`, a high performance Python compiler
`Numba` is an open source, **Just In Time** compiler that translates a subset of `Python` and `NumPy` code into fast machine code using the `LLVM` compiler library.

`Numba` provides a `jit` decorator to indicate the function should be compiled. It usually performs better on explicit loops but accepts most of the `NumPy` constructions. The `jit` decorator takes those options:
* signature of the function
* `nogil=True`: the function should not contain any Python construct after compilation
* `nopython=True`: raises an exception when Python construct remains
* `parallel=True`: enables parallel excution with `numba.prange` instead of `range`. Requires `nogil` and `nopython`

**Disclaimer:** With any of those options, you leave the cosy world of Python and enter the `real life`: any bug in your code (or in the compiler) can lead to a `segmentation fault` which appears like a `dead kernel` in Jupyter. If this occures, restart at the cell before numexpr to load all parameters for the tutorial.

In [ ]:
import numba as nb

#  Limit the number of cores to be used
try:
    # On recent version on numba: 
    nb.set_num_threads(n_cpu)
except:
    import os
    os.environ["NUMBA_NUM_THREADS"] = str(n_cpu) 




In [ ]:
@nb.jit
def test_one(a):
    result = np.zeros(a.shape)
    for i in range(a.size):
        result[i] = np.cos(a[i])**2 + np.sin(a[i])**2
    return result

a = np.arange(10)
%time test_one(a)
%timeit test_one(a)
test_one(a)

In [ ]:
# Get the assembly code
for key, value in test_one.inspect_asm().items():
    print("Signature:", key)
    print("Machine code:\n",value)

### Square crystal implemented with `Numba`

In [ ]:
@nb.jit(parallel=True)
def laue_numba(N, h, k):
    result = np.zeros((h.size, k.size), dtype=np.float64)
    for i in nb.prange(h.size):  # loop over the reciprocal space coordinates
        for j in range(k.size):
            tmp = 0j
            for n in range(N):  # loop and sum over unit-cells
                for m in range(N):
                    tmp +=  cmath.exp(2j*np.pi*(h[i]*n + k[j]*m))
            result[i, j] = abs(tmp)**2
    return result

In [ ]:
#Compute and check error
%time intensity = laue_numba(N, h, k)
print("Error:", validate_sq(intensity))
perf_sq_numba = %timeit -o laue_numba(N, h, k)

In [ ]:
%memit laue_numba(N, h, k)

### Exercise

1. Implement the strained crystal diffraction using `Numba` ([Documentation](https://numba.readthedocs.io/)).
2. Profile the code and store the timings in `perf_ci_numba`.

In [ ]:
from solution import circ_numba   # Replace with your implementation
# @nb.jit(parallel=True)
# def circ_numba(N, h, k):
#     ...

In [ ]:
%time intensity = circ_numba(N, h, k)
print("Error:", validate_ci(intensity))
perf_ci_numba = %timeit -o  circ_numba(N, h, k)

### Limits of `Numba`
`Numba` being a JIT compiler, it requires the compiler `LLVM-lite` to be installed and configured on every single node.

The second drawback is the JIT: the compilation will be performed for the first execution of a session and if it runs on a cluster, the compilation will occur on every node. Numba is not `multi-processing` friendly.

Finally the support for multi-threading in classes looks brittle, at least when evaluated.

## Cython: C-extensions for Python

Static, ahead-of-time compiler for Python: compiles Python code into C-extensions (or C++ if you wish).

Normal Python code gets usually 30% speed-up but for numerical kernels, native C-speed is achievable easily.
To get substential speed-up, one need to declare most variables and explicitly write loops. 

The methodology is pretty similar to `numba`:
1. Write the code with loops
2. Use the `cython -a` to annotate the code to highlight potential hot-spots, in yellow.
3. Declare variables which are not infered with `cdef` and the proper type
4. Isolate the numerical kernel and replace numpy functions with the ones provided by the `libc` or `libm` which are guaranteed to be GIL-free.

In [ ]:
import os

os.environ["OMP_NUM_THREADS"] = str(n_cpu)
# This enables the %cython mode
%load_ext Cython

### Square crystal implemented with `Cython`

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp -a
#cython: embedsignature=True, language_level=3, binding=True
#cython: boundscheck=False, wraparound=False, cdivision=True, initializedcheck=False,
## This is for development:
## cython: profile=True, warn.undeclared=True, warn.unused=True, warn.unused_result=False, warn.unused_arg=True

import numpy as np
from cython.parallel import prange

# With Cython3: from libc.complex cimport cabs, cexp
# Accessing C code from cython (out of the scope for today)
cdef extern from "complex.h" nogil:
    double cabs(double complex)
    double complex cexp(double complex)


def laue_cython(int N, 
                double[::1] h, 
                double[::1] k):
    cdef:
        double[:, ::1] result
        double complex tmp, two_j_pi
        int i_h, i_k, m, n, h_size, k_size
        
    two_j_pi = np.pi*2j
    h_size = h.shape[0]
    k_size = k.shape[0]
    result = np.zeros((h_size, k_size))
    
    for i_h in prange(h_size, nogil=True):  # loop over the reciprocal space coordinates
        for i_k in range(k_size):
            tmp = 0.0
            for n in range(N):  # loop and sum over unit-cells
                for m in range(N):
                    tmp += cexp(two_j_pi*(h[i_h]*n + k[i_k]*m))
            result[i_h, i_k] += cabs(tmp)**2
    return np.asarray(result)

In [ ]:
#Compute and check error
%time intensity = laue_cython(N, h, k)
print("Error:", validate_sq(intensity))
perf_sq_cython = %timeit -o laue_cython(N, h, k)

### Exercise

1. Implement the strained crystal diffraction using `Cython` ([Documentation](http://docs.cython.org/en/latest/)).

2. Profile the code and store the timings in `perf_ci_cython`.

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp -a
#%%cython -a
#cython: embedsignature=True, language_level=3, binding=True
#cython: boundscheck=False, wraparound=False, cdivision=True, initializedcheck=False,
## This is for development:
## cython: profile=True, warn.undeclared=True, warn.unused=True, warn.unused_result=False, warn.unused_arg=True

import numpy as np
from cython.parallel import prange
from libc.math cimport sqrt, pi, tanh

# With Cython3: from libc.complex cimport cabs, cexp
# Accessing C code from cython (out of the scope for today)
cdef extern from "complex.h" nogil:
    double cabs(double complex)
    double complex cexp(double complex)
    
def circ_cython(int N, 
                double[::1] h, 
                double[::1] k,
                double e0,
                double w):
    return np.ones((h.size, k.size))  # Replace with your implementation

In [ ]:
#Compute and check error
%time intensity = circ_cython(N, h, k, e0, w)
print("Error:", validate_ci(intensity))
perf_ci_cython = %timeit -o circ_cython(N, h, k, e0, w)

### Limits of `Cython`

* The learning curve of `Cython` is steeper than the one of `numba` since it targets C/C++ as intermediate language.
* Distribution of compiled extensions can be an issue. 
* Distribution of source code (C or C++) requires the user to install a compiler (rarely on consumer targeted operating systems like Windows & MacOS).
* Compiler features depend a lot on the operating system: for instance MacOS removed the support for OpenMP.
* Issues about software distribution are addressed in the software engineering training course.

The issue about OpenMP not being supported in some environments can be worked around using a `ThreadPool` in Python and a `nogil` section in Cython:

In [ ]:
%%cython -a
#cython: embedsignature=True, language_level=3, binding=True
#cython: boundscheck=False, wraparound=False, cdivision=True, initializedcheck=False,
## This is for development:
## cython: profile=True, warn.undeclared=True, warn.unused=True, warn.unused_result=False, warn.unused_arg=True

from libc.math cimport pi

# With Cython3: from libc.complex cimport cabs, cexp
# Accessing C code from cython (out of the scope for today)
cdef extern from "complex.h" nogil:
    double cabs(double complex)
    double complex cexp(double complex)

    
def laue_nogil(int N, 
               double h, 
               double k):
    cdef:
        double result
        double complex tmp, two_j_pi
        int m, n
    
    with nogil:
        two_j_pi = pi*2j    
        tmp = 0.0
        for n in range(N):  # loop and sum over unit-cells
            for m in range(N):
                tmp += cexp(two_j_pi*(h*n + k*m))
        result = cabs(tmp)**2
    return result

In [ ]:
from multiprocessing.pool import ThreadPool
from itertools import product

def laue_pool(N, h, k):
    def local_laue(i, j):
        return laue_nogil(N, i, j)
    with ThreadPool(n_cpu) as pool:
        res = pool.starmap(local_laue, product(h, k))
    return np.array(res).reshape(h.size, k.size)

#Compute and check error
%time intensity = laue_pool(N, h, k)
print("Error:", validate_sq(intensity))
perf_sq_pool = %timeit -o laue_pool(N, h, k) 

## Other players in the game of optimizing Python code

This tutorial demonstrated 4 ways of optimizing:
* Vectorization with `NumPy`
* Numerical kernel evaluation with `NumExpr`
* Just In Time (JIT)-compilation with `Numba`
* Ahead Of Time (AOT)-compilation with `Cython`

There are several other ways to optimize code:
* `Pythran` which is a Python to C++ compiler (with parallel and SIMD vectorization)
* `Transonic` which is a wrapper over the different methods exposed here
* Write Fortran code and bind it with `F2py`
* Write C++ code and bind it with Cython, Boost, pybind11, ...
* Write C code and bind it with Cython, ctypes, swig, ...

Last but not least, the best way to optimize the code is to use the proper algorithm. 
In this example, the diffraction of the square crystal can be calculated using a FFT.

### Use the right algorithm
The best way to optimize the code is to use the proper algorithm. In this example, the diffraction of the square crystal can be calculated using a FFT:

In [ ]:
def laue_fft(N, h, k):
    support = np.zeros([h.size,k.size]) #create a support function padded with 0s
    support[0:N, 0:N]=1 #the first N values
    return np.fft.fftshift(abs(np.fft.fft2(support))**2)

%time intensity = laue_fft(N, h, k)
print("Error:", validate_sq(intensity))
perf_sq_fft = %timeit -o laue_fft(N, h, k)

Unfortunately, the `FFT` approach does not work for strained crystals.

## Warning about parallel programming

This tutorial presented you several tools for doing parallel processing. Keep in mind parallel processing is **complicated** and many pitfalls exist. We have focused on `Map`, also called `Element-wise` numerical kernel which are well addressed by the `prange` approach. Beside this, risks are great that if you calculate the index position in the loop that you end up with conflicting writes (and incorrect results).

**[Amdahl's law](https://en.wikipedia.org/wiki/Amdahl%27s_law):**
Profile your full application and search for bottlenecks. If you numerical kernel is acounting for 80% of the time, a complete parallelization of it will lead to a speed-up of a factor five, at best.  You have been warned!

![Amdahl's Law](https://upload.wikimedia.org/wikipedia/commons/e/ea/AmdahlsLaw.svg)



## Conclusions

Within the Python ecosystem, the `NumPy` library is the *de facto* standard when it comes to scientific
computing. As long as algorithms are properly vectorized and memory is large enough to store
arrays, it allows to reach high computational performances while keeping a clean and simple code,
close to mathematical notation. Used in combination with `NumExpr` library, simple `NumPy` code
can benefit from multi-core CPUs as well as optimized memory management, with very little code
modification.

In the case where it is not possible to vectorize algorithms, or when increased performances are
critical, one must make use of compilers that translate Python code into statically-typed code that also
provide an improved support of multi-core architectures. We have shown that `Numba` and `Cython` in
general exhibit very close performances and, given the heavier syntax of `Cython`, `Numba` is easier to
implement. `Cython`, on the other hand, allows to access more advanced options regarding threads, 
memory management and debugging.

### Summary of the execution runtimes

In [ ]:
print("Runtime   Square (ms)  Speed-up (x)")
ref = perf_sq_numpy.best
print(f"Python    {1000*perf_sq_python.best:6.1f} ms   {ref/perf_sq_python.best:6.3f}x")
print(f"Numpy     {1000*perf_sq_numpy.best:6.1f} ms   {ref/perf_sq_numpy.best:6.3f}x")
print(f"NumExpr   {1000*perf_sq_numexpr.best:6.1f} ms   {ref/perf_sq_numexpr.best:6.3f}x")
print(f"Numba     {1000*perf_sq_numba.best:6.1f} ms   {ref/perf_sq_numba.best:6.3f}x")
print(f"Cython    {1000*perf_sq_cython.best:6.1f} ms   {ref/perf_sq_cython.best:6.3f}x")
print(f"Pool      {1000*perf_sq_pool.best:6.1f} ms   {ref/perf_sq_pool.best:6.3f}x")
print(f"FFT       {1000*perf_sq_fft.best:6.1f} ms   {ref/perf_sq_fft.best:6.1f}x")

In [ ]:
print("Runtime    Circle (ms)  Speed-up (x)")
ref = perf_ci_numpy.best
print(f"Python  {1000*perf_ci_python.best:8.1f} ms  {ref/perf_ci_python.best:6.3f}x")
print(f"Numpy   {1000*perf_ci_numpy.best:8.1f} ms  {ref/perf_ci_numpy.best:6.3f}x")
print(f"NumExpr {1000*perf_ci_numexpr.best:8.1f} ms  {ref/perf_ci_numexpr.best:6.3f}x")
print(f"Numba   {1000*perf_ci_numba.best:8.1f} ms  {ref/perf_ci_numba.best:6.3f}x")
print(f"Cython  {1000*perf_ci_cython.best:8.1f} ms  {ref/perf_ci_cython.best:6.3f}x")
# print(f"Pool    {1000*perf_sq_pool.best:8.1f} ms  {np.NaN:6.3f}x")

|               | multithreaded |  vector programming | GIL free | Compilation              |
|---------------|:-------------:|:-------------------:|:--------:|:------------------------:|
| "pure" python | X             | X                   | X        |  interpreted  |
| numpy         | (X)           | V                   | (V)      |  interpreted  |
| numexpr       | V             | V                   | V        |  just in time |
| numba         | V             | (V)                 | V        |  just in time |
| cython        | V             | (V)                 | V        | ahead of time |